In [3]:
pip install torch torch-geometric scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

In [6]:
# Step 1: Generate synthetic data
num_nodes = 100
num_timesteps = 10  # Number of time steps for temporal data

# Generate node features (e.g., traffic_load, energy_level, etc.)
node_features = np.random.rand(num_nodes, num_timesteps, 2)  # 2 features per node per timestep

# Generate edge indices (graph structure)
edge_index = torch.tensor([[i, j] for i in range(num_nodes) for j in range(num_nodes) if i != j], dtype=torch.long).t().contiguous()

# Generate labels (e.g., node_state: 0=active, 1=inactive, 2=sleep)
labels = np.random.choice([0, 1, 2], num_nodes)


In [7]:
# Step 2: Convert data to PyTorch Geometric format
# Create a list of Data objects for each timestep
data_list = []
for t in range(num_timesteps):
    x = torch.tensor(node_features[:, t, :], dtype=torch.float)  # Node features at timestep t
    y = torch.tensor(labels, dtype=torch.long)  # Labels
    data = Data(x=x, edge_index=edge_index, y=y)
    data_list.append(data)

In [18]:
# Step 3: Define GCN + LSTM model
class GCNLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(GCNLSTM, self).__init__()
        self.gcn = GCNConv(input_dim, hidden_dim)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x_list, edge_index):
        # Apply GCN to each timestep
        h_list = []
        for x in x_list:
            # Ensure x is a PyTorch tensor with the correct dtype
            x = x.type(torch.float)  # Convert x to torch.float32
            h = self.gcn(x, edge_index)
            h = F.relu(h)
            h_list.append(h.unsqueeze(0))  # Add batch dimension
            # Stack GCN outputs and pass through LSTM
        h_stack = torch.cat(h_list, dim=0)  # Shape: (timesteps, num_nodes, hidden_dim)
        h_stack = h_stack.permute(1, 0, 2)  # Reshape for LSTM: (num_nodes, timesteps, hidden_dim)
        _, (h_n, _) = self.lstm(h_stack)  # h_n: (1, num_nodes, hidden_dim)
        h_n = h_n.squeeze(0)  # Remove batch dimension

        # Final classification
        out = self.fc(h_n)
        return out

In [19]:
# Step 4: Initialize model, optimizer, and loss function
input_dim = 2  # Number of features per node
hidden_dim = 16  # Hidden dimension for GCN and LSTM
num_classes = 3  # Number of classes (active, inactive, sleep)

model = GCNLSTM(input_dim, hidden_dim, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()


In [20]:
# Step 5: Train the model
def train(data_list, edge_index, labels, epochs=50):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data_list, edge_index)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch + 1}, Loss: {loss.item()}")


In [21]:
# Step 6: Evaluate the model
def evaluate(data_list, edge_index, labels):
    model.eval()
    with torch.no_grad():
        out = model(data_list, edge_index)
        preds = out.argmax(dim=1)
        accuracy = accuracy_score(labels.numpy(), preds.numpy())
        print(f"Accuracy: {accuracy * 100:.2f}%")

In [22]:
print(train)
print(evaluate)


<function train at 0x7b5fbf0ce0c0>
<function evaluate at 0x7b5fbf0ccb80>


In [23]:
print(type(data_list))
print(type(edge_index))
print(type(labels))


<class 'list'>
<class 'torch.Tensor'>
<class 'numpy.ndarray'>


In [24]:
# Step 7: Train and evaluate
train (data_list, edge_index, torch.tensor(labels, dtype=torch.long))
evaluate(data_list, edge_index, torch.tensor(labels, dtype=torch.long))


AttributeError: 'GlobalStorage' object has no attribute 'type'